# Tutorial

This is the SQLAlchemy Tutorial found in: http://docs.sqlalchemy.org/en/latest/orm/tutorial.html

## Connecting

In [3]:
import sqlalchemy as sql

To connect to a Database, use the `create_engine` function

In [4]:
engine = sql.create_engine('sqlite:///:memory:', echo=True)

## Declaring a Mapping

The `declarative` system allows to create classes that include direcetives to describe the actual database table they will be mapped to - DB descrption + Class definition

In [5]:
from sqlalchemy.ext.declarative import declarative_base

In [6]:
base = declarative_base()

If the class is using base, it requires at a minimum:
* `__tablename__`
* One Column with primary key

In [7]:
from sqlalchemy import Column, Integer, String
class User(base):
    __tablename__ = 'users'
    
    id = Column(Integer,primary_key=True)
    name = Column(String)
    #name = Column(String(50)) --> To provide the length when VARCHAR requires it
    fullname = Column(String)
    password = Column(String)
    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (
            self.name, self.fullname, self.password)

## Schema Creation

In [8]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('password', String(), table=<users>), schema=None)

In [9]:
base.metadata.create_all(engine)

2017-06-29 12:19:05,890 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-06-29 12:19:05,910 INFO sqlalchemy.engine.base.Engine ()
2017-06-29 12:19:05,911 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-06-29 12:19:05,914 INFO sqlalchemy.engine.base.Engine ()
2017-06-29 12:19:05,917 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2017-06-29 12:19:05,918 INFO sqlalchemy.engine.base.Engine ()
2017-06-29 12:19:05,921 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2017-06-29 12:19:05,923 INFO sqlalchemy.engine.base.Engine ()
2017-06-29 12:19:05,924 INFO sqlalchemy.engine.base.Engine COMMIT


## Creating an Instance

In [10]:
end_user = User(name = 'Juan',fullname = 'Juan Daza', password = 'password')

In [11]:
end_user.name

'Juan'

In [12]:
str(end_user.id)

'None'

## Creating a Session

In [13]:
from sqlalchemy.orm import sessionmaker

In [14]:
Session = sessionmaker(bind = engine)

In [15]:
session = Session()

In [16]:
session.add(end_user)

In [17]:
user = session.query(User).filter_by(name = 'Juan').first()

2017-06-29 12:19:06,108 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-06-29 12:19:06,112 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-06-29 12:19:06,115 INFO sqlalchemy.engine.base.Engine ('Juan', 'Juan Daza', 'password')
2017-06-29 12:19:06,125 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2017-06-29 12:19:06,128 INFO sqlalchemy.engine.base.Engine ('Juan', 1, 0)


In [18]:
print user

<User(name='Juan', fullname='Juan Daza', password='password')>


In [19]:
user == end_user

True

In [20]:
session.add_all([User(name='John', fullname='John Doe', password='password'),
                User(name='Charles', fullname='Charles Xavier', password='password'),
                User(name='Logan', fullname='Logan Xavier', password='password')])

In [21]:
user.password = 'change1'

In [22]:
user = session.query(User).filter_by(name = 'Juan').first()

2017-06-29 12:19:06,223 INFO sqlalchemy.engine.base.Engine UPDATE users SET password=? WHERE users.id = ?
2017-06-29 12:19:06,224 INFO sqlalchemy.engine.base.Engine ('change1', 1)
2017-06-29 12:19:06,226 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-06-29 12:19:06,229 INFO sqlalchemy.engine.base.Engine ('John', 'John Doe', 'password')
2017-06-29 12:19:06,232 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-06-29 12:19:06,233 INFO sqlalchemy.engine.base.Engine ('Charles', 'Charles Xavier', 'password')
2017-06-29 12:19:06,236 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-06-29 12:19:06,236 INFO sqlalchemy.engine.base.Engine ('Logan', 'Logan Xavier', 'password')
2017-06-29 12:19:06,240 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS u

In [23]:
print user

<User(name='Juan', fullname='Juan Daza', password='change1')>


In [24]:
end_user.password = 'change2'

In [25]:
user = session.query(User).filter_by(name = 'Juan').first()

2017-06-29 12:19:06,299 INFO sqlalchemy.engine.base.Engine UPDATE users SET password=? WHERE users.id = ?
2017-06-29 12:19:06,302 INFO sqlalchemy.engine.base.Engine ('change2', 1)
2017-06-29 12:19:06,306 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2017-06-29 12:19:06,319 INFO sqlalchemy.engine.base.Engine ('Juan', 1, 0)


In [26]:
print user

<User(name='Juan', fullname='Juan Daza', password='change2')>


In [27]:
session.dirty

IdentitySet([])

In [28]:
session.new

IdentitySet([])

In [29]:
session.commit()

2017-06-29 12:19:06,404 INFO sqlalchemy.engine.base.Engine COMMIT


In [30]:
user.id

2017-06-29 12:19:06,420 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-06-29 12:19:06,421 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?
2017-06-29 12:19:06,426 INFO sqlalchemy.engine.base.Engine (1,)


1

## Rolling Back

In [31]:
fake_user = User(name='fakeuser', fullname='Invalid', password='12345')
session.add(fake_user)

In [32]:
session.commit()

2017-06-29 12:19:06,479 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-06-29 12:19:06,486 INFO sqlalchemy.engine.base.Engine ('fakeuser', 'Invalid', '12345')
2017-06-29 12:19:06,490 INFO sqlalchemy.engine.base.Engine COMMIT


The following rollback will not work as the commit command will already be executed - The DB requires now that the user is deleted

In [33]:
session.rollback()

In [34]:
fake_user in session

True

In [35]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

2017-06-29 12:19:06,579 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-06-29 12:19:06,584 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)
2017-06-29 12:19:06,585 INFO sqlalchemy.engine.base.Engine ('Edwardo', 'fakeuser')


[<User(name='fakeuser', fullname='Invalid', password='12345')>]

In [36]:
rollback_user = User(name='fakeuser', fullname='Invalid', password='12345')

In [37]:
session.add(rollback_user)

In [38]:
session.query(User).filter(User.name.in_(['fakeuser'])).all()

2017-06-29 12:19:06,654 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-06-29 12:19:06,660 INFO sqlalchemy.engine.base.Engine ('fakeuser', 'Invalid', '12345')
2017-06-29 12:19:06,670 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?)
2017-06-29 12:19:06,674 INFO sqlalchemy.engine.base.Engine ('fakeuser',)


[<User(name='fakeuser', fullname='Invalid', password='12345')>,
 <User(name='fakeuser', fullname='Invalid', password='12345')>]

In [39]:
session.rollback()

2017-06-29 12:19:06,698 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [40]:
rollback_user in session

False

# Relationships

http://docs.sqlalchemy.org/en/latest/orm/basic_relationships.html#relationship-patterns

In [41]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

In [44]:
class Address(base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String,nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))
    
    user = relationship('User',back_populates='addresses')
    
    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address

In [46]:
base.metadata.create_all(engine)

2017-06-29 12:22:04,846 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2017-06-29 12:22:04,848 INFO sqlalchemy.engine.base.Engine ()
2017-06-29 12:22:04,851 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2017-06-29 12:22:04,852 INFO sqlalchemy.engine.base.Engine ()
2017-06-29 12:22:04,855 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE addresses (
	id INTEGER NOT NULL, 
	email_address VARCHAR NOT NULL, 
	user_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES users (id)
)


2017-06-29 12:22:04,857 INFO sqlalchemy.engine.base.Engine ()
2017-06-29 12:22:04,859 INFO sqlalchemy.engine.base.Engine COMMIT
